# Chapter 10 - Using Subclassing API to Build Dynamic Models (pg. 313)

In [1]:
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
print(keras.__version__)

2.0.0
2.2.4-tf


__Benefits__ of subclassing the `Model` class:
- more flexibility to do for loops, if statements, low-level Tensorflow operations, etc.. in the `call()` method. This is great for research.

__Disadvantages:__
- the model's architecture is hidden within the `call()` method, so Keras cannot easily inspect it, it cannot save or clone it, and when `summary()` method is called it can only display a list of layers without any information on how they are connected to each other.
- Keras also cannot check types and shapes ahead of time, and it is easier to make mistakes.
- __So, unless flexibility is really needed, just stick to either Sequential or Function APIs__

In [ ]:
class WideAndDeepModel(tf.keras.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs) # handles standard args (e.g., name)
        self.hidden1 = tf.keras.Dense(units, activation=activation)
        self.hidden2 = tf.keras.Dense(units, activation=activation)
        self.main_output = tf.keras.layers.Dense(1)
        self.aux_output = tf.keras.layers.Dense(1)
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = tf.keras.layers.concatenate([input_A, hidden2])
    
        # name can't be simply "output" (will conflict with Keras 'output' attribute)
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output
    
model = WideAndDeepModel() # similar to Functional API now